# Capstone Project Notebook

#### Import required packages:

In [1]:
import pandas as pd
import numpy as np

In [2]:
# !pip install bs4
# !pip install requests
from bs4 import BeautifulSoup
import requests

### PartA: Obtaining information from wikipedia

In [3]:
#Mark the url and obtain lxml
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url).text
soup = BeautifulSoup(req,'lxml')

In [4]:
#cut out the required table
table = soup.find('table',{'class':'wikitable sortable'})
ths = table.find_all('th')

In [5]:
#Clean the table data and put into a dataframe
rows = table.findAll("tr")
row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
ncols = max(row_lengths)
nrows = len(rows)
data = []
for i in range(nrows):
    rowD = []
    for j in range(ncols):
        rowD.append('')
    data.append(rowD)

for i in range(len(rows)):
    row = rows[i]
    rowD = []
    cells = row.findAll(["td", "th"])
    for j in range(len(cells)):
        cell = cells[j]

        #lots of cells span cols and rows so lets deal with that
        cspan = int(cell.get('colspan', 1))
        rspan = int(cell.get('rowspan', 1))
        l = 0
        for k in range(rspan):
            # Shifts to the first empty cell of this row
            while data[i + k][j + l]:
                l += 1
            for m in range(cspan):
                cell_n = j + l + m
                row_n = i + k
                # in some cases the colspan can overflow the table, in those cases just get the last item
                cell_n = min(cell_n, len(data[row_n])-1)
                data[row_n][cell_n] += cell.text

    data.append(rowD)

In [6]:
df = pd.DataFrame(data, columns=["Postcode", "Borough", "Neighbourhood"])
#Remove no post code records
df = df[df.Postcode != 'Postcode']
#drop null values
df = df.dropna()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
#Clean the Neighbourhood column
df = df.replace('\n', '', regex=True)
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df['Borough']

In [46]:
#Remove duplicate by merging
df_dup = df[df.duplicated(subset=["Postcode", "Borough"],keep=False)].groupby(["Postcode", "Borough"])['Neighbourhood'].apply(','.join).reset_index()
df_nodup = df[~df.duplicated(subset=["Postcode", "Borough"],keep=False)]
df_final = pd.concat([df_dup, df_nodup], ignore_index=True, sort = False)

In [47]:
#Checking if there are missing Postcode
print(df_final.Postcode.nunique(),df.Postcode.nunique())

103 103


In [48]:
df_final.shape

(103, 3)

### PartB: Obtain geographical coordinates

In [12]:
# #!pip install geocoder
# import geocoder # import geocoder
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [13]:
# Geocoder not working
# for index, row in df.iterrows():
#     g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
#     lat_lng_coords = g.latlng
#     print(lat_lng_coords)

In [14]:
# Geocoder not working, use provided data instead
pd_geo = pd.read_csv('http://cocl.us/Geospatial_data')
pd_geo.head()

In [22]:
df_lat_lng = pd.merge(df_final, pd_geo, left_on='Postcode', right_on='Postal Code')
df_lat_lng.drop(['Postal Code'], axis = 1)
df_lat_lng.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353
1,M1B,Scarborough,Rouge,M1B,43.806686,-79.194353
2,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353
3,M1C,Scarborough,"Rouge Hill,Port Union",M1C,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,M1C,43.784535,-79.160497


In [25]:
df_lat_lng.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353
1,M1B,Scarborough,Rouge,M1B,43.806686,-79.194353
2,M1B,Scarborough,Malvern,M1B,43.806686,-79.194353
3,M1C,Scarborough,"Rouge Hill,Port Union",M1C,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,M1C,43.784535,-79.160497


In [43]:
df[df['Postcode'] == 'M1B']

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [41]:
df_dup[df_dup['Postcode'] == 'M1B']

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"


In [42]:
df_nodup[df_nodup['Postcode'] == 'M1B']

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Queen's Park
14,M3B,North York,Don Mills North
19,M6B,North York,Glencairn
33,M4C,East York,Woodbine Heights
34,M5C,Downtown Toronto,St. James Town
35,M6C,York,Humewood-Cedarvale
